In [1]:
import os
import numpy as np
import pandas as pd

from LHS_sequential_sampling import LHS_sequential_sampling
#from Surrogate_model_build import Surrogate_model_build

import win32com.client as client



In [2]:
class SimulationInput:
    def __init__(self):
        self.savesetting = 2
        # 0 : 샘플링 진행 아무것도 안된 처음상태
        # 1 : 초기 LHS 상태 저장 완료
        # 2 : Final sampling DATA 존재
        # 3 : 샘플링 전체 완료. serult 추출 중 정지
        
        self.opt_variable_name = ['DMAG3', 'MT1', 'MT3', 'MT4', 'MT5', 'MW1', 'MW3', 'WMAG1', 'WMAG3', 'WMAG5']
        self.opt_variable_upper = np.array([15, 14, 14, 14, 14, 40, 26, 15, 15, 20])
        self.opt_variable_lower = np.array([5, 5, 5, 5, 5, 25, 12, 5, 10, 10])

        # Efficiency Map Variable
        self.SLOTS = 36
        self.POLE = 6
        self.RPM = 4500
        self.HEIGHT = 285
        self.MECHANICAL_LOSS = 20
        self.CURRENT_SOURCE = 353.5533906
        self.COILEND_H = 110
        self.VOLTAGE_SOURCE = 3086.448943904834

        # Heat Analysis Variable
        self.AIR_HTC = 10
        self.AIR_TC = 0.02
        self.COIL_TC = 400
        self.COIL_DENS = 8.96
        self.COIL_SH = 380
        self.SHAFT_TC = 25
        self.SHAFT_DENS = 7.8
        self.SHAFT_SH = 450
        self.HOUSING_H = 420
        self.HOUSING_T = 20
        self.HOUSING_TC = 25
        self.HOUSING_DENS = 7.8
        self.HOUSING_SH = 450

        self.AMBIENT = 70

        # Load Analysis Settings
        self.OPERATING_CURRENT = 200
        self.MAGNET_TEMP = 180
        # OPERATING_PHASE는 다변수 최적화에 사용되므로 여기에 포함되지 않습니다.
        self.OPERATING_RPM = 4387
        self.OPERATING_TRQ = 923


        self.study_num = 7 #Analysis Group내의 전체 스터디 갯수

        ##### 아래 두 변수는 시퀀셜 LHS샘플링을 위한 변수들임.
        self.lhs_sample_num = 4000 # 전체 샘플 갯수
        self.lhs_sample_division = int(self.lhs_sample_num/40) #sample_num을 몇개의 jmag 프로젝트 파일로 분할 할지
        ''' 즉, 한 iter에 sample_num만큼 데이터를 뽑는데 sample_division만큼의 프로젝트 파일에 
            각 sample_num/sample_division 만큼의 case를 생성시켜서 해석함.

            ex) self.lhs_sample_num = 400 / self.lhs_sample_division = 40
            한 iteration당 400개 샘플 생성 / 프로젝트 40개 / 프로젝트당 10개 case
        '''
        self.lhs_iteration_num = 20 # sampling iteration(시퀀셜)을 최대 몇번 진행할지
        
        # => 총 생성될 data는 iteration num * smple_num 임.

        #효율맵 최적화 목적함수를 위한 운전점(OP)
        self.op = [[1194, 3267],[4387, 923]] #[[rpm, torque],[rpm, torque],[rpm, torque].....]
        
        self.analysis_type_all = ['Machine Characteristics (Efficiency Map)','Cogging Torque','Torque Ripple','Induced Voltage','Demagnetization','Natural/Forced (Steady)/','Stress']
        self.analysis_type_selection = ['Machine Characteristics (Efficiency Map)','Cogging Torque','Torque Ripple','Induced Voltage']

        #Surrogate Model 선택
        self.model_types = ['linear_regression', 'polynomial_regression', 'gaussian_process', 'svm', 'decision_tree', 'random_forest', 'gradient_boosting', 'neural_network', 'xgboost', 'lightgbm', 'catboost', 'knn', 'elastic_net', 'kriging']
        
sim_input = SimulationInput()
print('Varialbe_Complete')

Varialbe_Complete


In [3]:
class Pathsetting:
    def __init__(self):
        # 현재 폴더
        self.code_dir = os.getcwd() # level 0
        self.parent_dir = os.path.dirname(self.code_dir)
        self.Parent_dir = os.path.dirname(self.parent_dir)
        self.current_dir = os.path.join(self.Parent_dir, 'Data')
        #self.current_dir = os.path.join(self.parent_dir, ',vscode')
        # input 샘플관련 하위 폴더 명
        self.casefolder_name = '01_Generated_Case' # level 1
        self.ini_LHS_samplefolder_name = '01_1st_LHS_INsamples' # level 2
        self.seq_LHS_samplefolder_name = '02_2nd_seqLHS_INsamples' # level 2
        self.seq_divsamplefolder_name = '03_divided_INsample' # level 2
        self.sample_tempfolder = '9999999_temp' # level 2
        

        # output 샘플관련 하위 폴더 명
        self.resultfolder_name = '02_Result' # level 1
        self.LHSoutputfolder_name = '01_LHS_Result_OUTsample' # level 2
        self.results_divfolder = '02_LHS_div_Result_OUTsample' # level 2
        self.results_tempfolder = '9999999_temp' # level 2

        # jproj 관련 하위 폴더 명
        self.jprojfolder_name = '03_Projectfile' # level 1
        self.divjprojfolder_name = '01_div_jproj' # level 2
        self.jproj_tempfolder = '9999999_temp' # level 2

        # 파일 명
        self.jmag_filename = '240718_EMU_Surro_OPT_After_SA.jproj'
        self.jmag_inputformat_filename = f'jmag_var_form.csv'
        self.Final_INsample_filename = f'Final_LHS_Seq_INsample.csv'  # 최종 input_seq_LHS 파일 명
        self.Final_OUTsample_filename = f'Final_LHS_Seq_OUTsample.csv'  # 최종 output_seq_LHS 파일 명

        # 주요 경로 명
        self.case_dir = os.path.join(self.current_dir, self.casefolder_name)
        self.result_dir = os.path.join(self.current_dir, self.resultfolder_name)
        self.projfolder_path = os.path.join(self.current_dir, self.jprojfolder_name)
        self.Final_INsample_filepath = os.path.join(self.case_dir, self.Final_INsample_filename)
        self.Final_OUTsample_filepath = os.path.join(self.result_dir, self.LHSoutputfolder_name, self.Final_OUTsample_filename)
        self.jmag_dir = os.path.join(self.current_dir, self.jprojfolder_name, self.jmag_filename)

pathset = Pathsetting()
print('Path Setting Complete')

Path Setting Complete


In [4]:
#03_After_SA_ver6/01_Generated_Case/jmag_var_form.csv 이경로에 case input 양식 파일이 위치해야함.
def main():
    sim_input = SimulationInput()
    pathset = Pathsetting()
    print(f'The number of sample is {sim_input.lhs_sample_num}')
    print(f'The number of J-mag projectfile is {sim_input.lhs_sample_division}')
    print(f'Maximum number of sequential sampling is {sim_input.lhs_iteration_num}')
    print(f"The selected surrogate model is {sim_input.model_types}.")
    print('**********************************************************************************************************************************************************************************')

    app = client.dynamic.Dispatch('designer.Application.230')
    scheduler = client.dynamic.Dispatch('scheduler.JobApplication.230')
    job = app.GetAnalysisGroup(0).CreateJob()

    #서로게이트 모델 빌드를 위한 라틴하이퍼큐브 시퀀셜 샘플링
    input, ouput = LHS_sequential_sampling(sim_input, pathset, app, job)

    #서로게이트 모델 빌드
    #Surrogate_model_build()
    #서로게이트 모델을 이용한 최적화
    #Optimization

    '''for i in range(sim_input.lhs_iteration_num):
        Final_LHS_filepath = os.path.join(gen_dir, f'Final_LHS_{i}.csv')
        lhs_samples_filepath = os.path.join(gen_dir, f'00_lhs_samples/lhs_samples_{i}.csv')
        output_filepath = os.path.join(result_dir, f'output_{i}.csv')
        results_filepath = os.path.join(result_dir, f'result_{i}.csv')

        #Final_LHS.csv 존재하면 LHS_Jmag_Run으로 이동
        if os.path.exists(Final_LHS_filepath):
            #print("The file {Final_LHS_filepath} already exists.")
            #Final_LHS.csv 존재하지 않으면 생성시작
            pass
        else:
            # 라틴하이퍼큐브(LHS) 샘플 생성 및 저장
            generateLHS.generateLHS(app,sim_input,lhs_samples_filepath,Final_LHS_filepath)

        #Jmag 실행 및 결과값 불러오기
        if os.path.exists(output_filepath):
            #print(f"The file 'output_{i}.csv' already exists.")
            parent_dir = os.path.dirname(current_dir)
            jmag_filepath = os.path.join(parent_dir, '02_Project_File', sim_input.jmag_filename)
            app.show()
            app.Load(jmag_filepath)
        else:
            LHS_Jmagsubmit(app,sim_input,Final_LHS_filepath)
            results = Jmagresultcheck(app,sim_input, output_filepath)
            resultsreforming(app, output_filepath, results_filepath)

        
    #input 최종 처리
    merge_csv_files(gen_dir, 'Final_LHS_', f'\Final_LHS_Combined.csv')
    merged_lhs_filepath = os.path.join(gen_dir, f'Final_LHS_Combined.csv')
    inputs_loader = input_var(merged_lhs_filepath)
    inputs = inputs_loader.get_multiple_columns_data(sim_input.opt_variable_name)

    #results 최종 처리
    #merge_csv_files(result_dir, 'output_', f'\output_Combined.csv')
    merge_csv_files(result_dir, 'result_', f'\\result_Combined.csv')
    outputs = pd.read_csv(result_dir+f'\\result_Combined.csv').to_dict('list')  
      
    #저장한 Final_LHS_results_inputs.csv 파일 불러와서 데이터 처리하기
    X = pd.DataFrame(inputs)
    Y = pd.DataFrame(outputs)

    # CSV 파일로 저장
    X.to_csv(result_dir+r'\01_Final_inputs_ouputs\inputs.csv', index=False)
    Y.to_csv(result_dir+r'\01_Final_inputs_ouputs\outputs.csv', index=False)
    #inputs, ouputs를 통한 surrogate model 구성
    # 해당 함수에서 모델의 설정(build_and_train_surrogate의 첫번 째 인수)은 진행 할 것
    input_output_path = os.path.join(result_dir,f'01_Final_inputs_ouputs')
    inputs_path = os.path.join(input_output_path, 'inputs.csv')
    outputs_path = os.path.join(input_output_path, 'outputs.csv')
    combined_path = os.path.join(input_output_path, 'inputs_outputs_combined.csv')
    merge_files(inputs_path, outputs_path, combined_path)
    modified_intput, modified_output = surrogatebuild.load_and_prepare_data(combined_path, sim_input)

    for model_type in sim_input.model_types:
        model, mse, r2 = surrogatebuild.build_and_train_surrogate(model_type, modified_intput, modified_output)
        print(f"{model_type} Model : MSE = {mse}, R² Score = {r2}")    
    '''
    # 생성된 surrogate Model의 정합성 평가
    ''' 정합성 평가의 방법은 다양함.
    1. 교차검증(Cross-Validation)
    2. 학습 곡선(Learning Curves)
    3. 평균 제곱 오차 및 R^2 점수 (Mean Squared Error (MSE) and R² Score)
    4. 통계적 테스트 (Statistical Testing)
    5. 모델 잔차 분석 (Model Residual Analysis)
    6. 기능 중요성 (Feature Importance)
    7. 외부 검증 (External Validation)'''

In [5]:
if __name__ == "__main__":
    main()

The number of sample is 4000
The number of J-mag projectfile is 100
Maximum number of sequential sampling is 20
The selected surrogate model is ['linear_regression', 'polynomial_regression', 'gaussian_process', 'svm', 'decision_tree', 'random_forest', 'gradient_boosting', 'neural_network', 'xgboost', 'lightgbm', 'catboost', 'knn', 'elastic_net', 'kriging'].
**********************************************************************************************************************************************************************************
Final sampling DATA Exist!
Final sampling DATA Exist and starting extraction of Jmag output results!
